Importing required Libraries

In [0]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import pandas_profiling

Connecting Google Drive with Colab

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Colab Notebooks/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks


Importing the dataset from Google drive to a dataframe (raw_data)

In [0]:
path='/content/drive/My Drive/Colab Notebooks/'
raw_data = pd.read_csv(path+'online_data.csv')

In [0]:
pandas_profiling.ProfileReport(raw_data)

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,18
Number of observations,12330
Total Missing (%),0.0%
Total size in memory,1.5 MiB
Average record size in memory,130.0 B
Numeric,13
Categorical,2
Boolean,2
Date,0
Text (Unique),0
Rejected,1


Removing NAs

In [0]:
raw_data = raw_data.dropna()

Encoding catagorical Variable 

In [0]:
from sklearn.preprocessing import LabelEncoder
#lable encoding for catagorical data
bin_cols = raw_data.nunique()[raw_data.nunique() < 11].keys().tolist()
le = LabelEncoder()
for i in bin_cols :
    raw_data[i] = le.fit_transform(raw_data[i])

Printing the clean dataset to see if the data is cleaned

In [7]:
raw_data

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0.0,0.0,0.0,0.0,1.0,0.000000,0.200000,0.200000,0.000000,0,2,0,1,0,1,2,0,0
1,0.0,0.0,0.0,0.0,2.0,64.000000,0.000000,0.100000,0.000000,0,2,1,2,0,2,2,0,0
2,0.0,-1.0,0.0,-1.0,1.0,-1.000000,0.200000,0.200000,0.000000,0,2,3,1,8,3,2,0,0
3,0.0,0.0,0.0,0.0,2.0,2.666667,0.050000,0.140000,0.000000,0,2,2,2,1,4,2,0,0
4,0.0,0.0,0.0,0.0,10.0,627.500000,0.020000,0.050000,0.000000,0,2,2,3,0,4,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12325,3.0,145.0,0.0,0.0,53.0,1783.791667,0.007143,0.029031,12.241717,0,1,3,6,0,1,2,1,0
12326,0.0,0.0,0.0,0.0,5.0,465.750000,0.000000,0.021333,0.000000,0,7,2,2,0,8,2,1,0
12327,0.0,0.0,0.0,0.0,6.0,184.250000,0.083333,0.086667,0.000000,0,7,2,2,0,13,2,1,0
12328,4.0,75.0,0.0,0.0,15.0,346.000000,0.000000,0.021053,0.000000,0,7,1,2,2,11,2,0,0


Splitting the data into Dependent and independent variables

In [0]:
X = raw_data.iloc[:, :-1].values
y = raw_data.iloc[:, -1].values

Creating a function for Performance Metrics

In [0]:
def performance_matrix(y_act, y_pred):
    from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, cohen_kappa_score, roc_curve,auc
    print("Accuracy = ", accuracy_score(y_act, y_pred))
    print("F1 Score = " ,f1_score(y_act, y_pred))
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_act, y_pred)
    print("AUC Score =", auc(false_positive_rate, true_positive_rate))
    print("Kappa score = ",cohen_kappa_score(y_act,y_pred))
    pass

Creating function for undersampling

In [0]:
def undersample(X, y): 
    from imblearn.under_sampling import RandomUnderSampler
    randomsample = RandomUnderSampler(random_state=2)
    X, y = randomsample.fit_resample(X, y)
    
    from sklearn.feature_selection import f_classif
    from sklearn.feature_selection import SelectKBest 
    fs = SelectKBest(f_classif, k = 8)
    X = fs.fit_transform(X, y)
    
    #Splitting the data into train test spit
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 50)
    
    from sklearn.naive_bayes import GaussianNB
    gnb = GaussianNB()
    gnb.fit(X_train, y_train)
        
    # Predicting the Test set results
    y_pred = gnb.predict(X_test)
    print("\n\nPerformance Matrix for Gaussian Naive Bayes:")
    performance_matrix(y_test, y_pred)
    
    from xgboost import XGBClassifier
    xgb = XGBClassifier(tree_method = 'gpu_hist')
    xgb.fit(X_train, y_train)   
    # Predicting the Test set results
    y_pred = xgb.predict(X_test)
    print("\n\nPerformance Matrix for XGBoost:")
    performance_matrix(y_test, y_pred)
    
    
    from sklearn.ensemble import RandomForestClassifier
    rf = RandomForestClassifier()
    rf.fit(X_train, y_train)
    # Predicting the Test set results
    y_pred = rf.predict(X_test)
    print("\n\nPerformance Matrix for RandomForest Classifier:")
    performance_matrix(y_test, y_pred)
    
    from sklearn.svm import SVC
    svc = SVC(random_state = 10)
    svc.fit(X_train, y_train)
    y_pred = svc.predict(X_test)
    print("\n\nPerformance Matrix for Support Vector Classifier:")
    performance_matrix(y_test, y_pred)

Initializing undersampling function

In [11]:
undersample(X, y)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)




Performance Matrix for Gaussian Naive Bayes:
Accuracy =  0.7617801047120419
F1 Score =  0.7465181058495821
AUC Score = 0.7666714755810199
Kappa score =  0.5275586270632445


Performance Matrix for XGBoost:
Accuracy =  0.8324607329842932
F1 Score =  0.8346253229974161
AUC Score = 0.8338726187286603
Kappa score =  0.6653136657654266


Performance Matrix for RandomForest Classifier:
Accuracy =  0.8259162303664922
F1 Score =  0.8310038119440915
AUC Score = 0.8264634556858542
Kappa score =  0.6516510339531567


Performance Matrix for Support Vector Classifier:
Accuracy =  0.643979057591623
F1 Score =  0.6784869976359338
AUC Score = 0.6402245076015197
Kappa score =  0.2820689988737416


Creating function for oversampling

In [0]:
def oversample(X, y): 
    from imblearn.over_sampling import SMOTE 
    sm = SMOTE() 
    X, y = sm.fit_sample(X, y)
    
    from sklearn.feature_selection import f_classif
    from sklearn.feature_selection import SelectKBest 
    fs = SelectKBest(f_classif, k = 8)
    X = fs.fit_transform(X, y)
    
    #Splitting the data into train test spit
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 50)
    
    from sklearn.naive_bayes import GaussianNB
    gnb = GaussianNB()
    gnb.fit(X_train, y_train)
        
    # Predicting the Test set results
    y_pred = gnb.predict(X_test)
    print("\n\nPerformance Matrix for Gaussian Naive bayes:")
    performance_matrix(y_test, y_pred)
    
    from xgboost import XGBClassifier
    xgb = XGBClassifier(tree_method = 'gpu_hist')
    xgb.fit(X_train, y_train)   
    # Predicting the Test set results
    y_pred = xgb.predict(X_test)
    print("\n\nPerformance Matrix for XGBoost Classifier:")
    performance_matrix(y_test, y_pred)
    
    
    from sklearn.ensemble import RandomForestClassifier
    rf = RandomForestClassifier(random_state = 2)
    rf.fit(X_train, y_train)
    # Predicting the Test set results
    y_pred = rf.predict(X_test)
    print("\n\nPerformance Matrix for RandomForest Classifier:")
    performance_matrix(y_test, y_pred)
    
    from sklearn.svm import SVC
    svc = SVC(random_state = 10)
    svc.fit(X_train, y_train)
    y_pred = svc.predict(X_test)
    print("\n\nPerformance Matrix for Support Vector Classifier:")
    performance_matrix(y_test, y_pred)

Initializing oversampling function

In [13]:
oversample(X, y)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)




Performance Matrix for Gaussian Naive bayes:
Accuracy =  0.7702161729383507
F1 Score =  0.7521160822249093
AUC Score = 0.769450228750795
Kappa score =  0.5396914265902188


Performance Matrix for XGBoost Classifier:
Accuracy =  0.9098478783026421
F1 Score =  0.9103074717221603
AUC Score = 0.9100406725068214
Kappa score =  0.8197439404057119


Performance Matrix for RandomForest Classifier:
Accuracy =  0.923138510808647
F1 Score =  0.9237853286757701
AUC Score = 0.9233736126212996
Kappa score =  0.8463305103861218


Performance Matrix for Support Vector Classifier:
Accuracy =  0.7213771016813451
F1 Score =  0.7294776119402985
AUC Score = 0.7218408284266459
Kappa score =  0.4432349482626551


Creating function for hyper-paramter tuning for XGBoost

In [0]:
def hyperparamter_tuning_XGB(X_train, y_train):
    from xgboost import XGBClassifier
    xgb = XGBClassifier(tree_method = 'gpu_hist')
    seed = 10
    est = range(1,100,5)
    eta = np.arange(0.01,0.2,0.05)
    params = {
            'min_child_weight': [1, 5, 10],
            'subsample': [0.6, 0.8, 1.0],
            'max_depth': [3, 4, 5],
            'n_estimators' : est,
            'learning_rate' : np.linspace(1e-16,1,3),
            'eta' : eta,
            
            }
    param_comb = 100
    
    from sklearn.model_selection import RandomizedSearchCV
    random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='accuracy', n_jobs= -1, verbose=3, random_state=10)
    random_search.fit(X_train, y_train)
    print('\n Best hyperparameters:')
    print(random_search.best_params_)

Creating function for hyper-paramter tuning for Randomforest

In [0]:
def hyperparamter_tuning_RF(X_train, y_train):
    from sklearn.ensemble import RandomForestClassifier
    rf = RandomForestClassifier()
    n_estimators = range(200, 2000, 10)
    max_depth = range(10, 110, 10)
    min_samples_leaf = [1, 2, 4]
    random_grid = {'n_estimators': n_estimators,
                   'max_depth': max_depth,
                   'min_samples_leaf': min_samples_leaf}
    from sklearn.model_selection import RandomizedSearchCV
    random_search = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
    random_search.fit(X_train, y_train)
    
    print('\n\n Best hyperparameters:')
    print(random_search.best_params_)

Creating function for hyper-paramter tuning for SVM

In [0]:
def hyperparamter_tuning_SVM(X_train, y_train):
    from sklearn.svm import SVC
    svm = SVC(random_state = 10)
    svm_params = {'C': range(1, 10, 1), 'gamma': np.arange(0.1, 1, 0.3)}
    
    from sklearn.model_selection import RandomizedSearchCV
    random_search = RandomizedSearchCV(estimator = svm, param_distributions = svm_params, n_iter = 10, cv = 5, verbose=2, random_state=42, n_jobs = -1)
    random_search.fit(X_train, y_train) 
    
    print('\n Best hyperparameters:')
    print(random_search.best_params_)

Creating function for undersampling with hyper-paramter tuning

In [0]:
def undersample_HP(X, y): 
    from imblearn.under_sampling import RandomUnderSampler
    randomsample = RandomUnderSampler(random_state=2)
    X, y = randomsample.fit_resample(X, y)
    
    from sklearn.feature_selection import f_classif
    from sklearn.feature_selection import SelectKBest 
    fs = SelectKBest(f_classif, k = 8)
    X = fs.fit_transform(X, y)
    
    #Splitting the data into train test spit
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 50)
    
    from sklearn.naive_bayes import GaussianNB
    gnb = GaussianNB()
    gnb.fit(X_train, y_train)
        
    # Predicting the Test set results
    y_pred = gnb.predict(X_test)
    print("\n\nPerformance Matrix for Gaussain Naive Bayes:")
    performance_matrix(y_test, y_pred)

    print("\n\n HyperParamter-Tuning XGBoost\n\n")
    from xgboost import XGBClassifier
    xgb = XGBClassifier(tree_method = 'gpu_hist', subsample = 0.8, n_estimators = 11, min_child_weight = 1, max_depth = 3, learning_rate = 0.5, eta = 0.060000000000000005)
    hyperparamter_tuning_XGB(X_train, y_train)
    xgb.fit(X_train, y_train)   
    # Predicting the Test set results
    y_pred = xgb.predict(X_test)
    print("\n\nPerformance Matrix for XGBoost Classifier with Hyper Paramter tuning:")
    performance_matrix(y_test, y_pred)
    
    
    print("\n\n HyperParamter-Tuning Random Forest \n\n")
    from sklearn.ensemble import RandomForestClassifier
    rf = RandomForestClassifier(n_estimators = 970, min_samples_leaf = 2, max_depth = 40)
    hyperparamter_tuning_RF(X_train, y_train)
    rf.fit(X_train, y_train)
    # Predicting the Test set results
    y_pred = rf.predict(X_test)
    print("\n\nPerformance Matrix for RandomForest Classifier with Hyper Paramter tuning:")
    performance_matrix(y_test, y_pred)
    
    print("\n\n HyperParamter-Tuning Support Vector Machine \n\n")
    from sklearn.svm import SVC
    hyperparamter_tuning_SVM(X_train, y_train)
    svc = SVC(random_state = 10, gamma = 0.1, C = 4)
    svc.fit(X_train, y_train)
    y_pred = svc.predict(X_test)
    print("\n\nPerformance Matrix for Support Vector Classifier with Hyper Paramter tuning:")
    performance_matrix(y_test, y_pred)

Initializing undersampling with hyper-paramter tuning function

In [18]:
undersample_HP(X, y)



Performance Matrix for Gaussain Naive Bayes:
Accuracy =  0.7617801047120419
F1 Score =  0.7465181058495821
AUC Score = 0.7666714755810199
Kappa score =  0.5275586270632445


 HyperParamter-Tuning XGBoost


Fitting 5 folds for each of 100 candidates, totalling 500 fits


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    7.5s
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:   22.5s
[Parallel(n_jobs=-1)]: Done 284 tasks      | elapsed:   47.1s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  1.3min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.



 Best hyperparameters:
{'subsample': 0.8, 'n_estimators': 6, 'min_child_weight': 10, 'max_depth': 4, 'learning_rate': 0.5, 'eta': 0.11}


Performance Matrix for XGBoost Classifier with Hyper Paramter tuning:
Accuracy =  0.8337696335078534
F1 Score =  0.834850455136541
AUC Score = 0.8355110845475843
Kappa score =  0.668144195909433


 HyperParamter-Tuning Random Forest 


Fitting 3 folds for each of 100 candidates, totalling 300 fits


/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 10.9min finished




 Best hyperparameters:
{'n_estimators': 800, 'min_samples_leaf': 2, 'max_depth': 10}


Performance Matrix for RandomForest Classifier with Hyper Paramter tuning:
Accuracy =  0.8285340314136126
F1 Score =  0.8331210191082802
AUC Score = 0.8292182766224934
Kappa score =  0.656978736821902


 HyperParamter-Tuning Support Vector Machine 


Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:    8.8s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   11.7s finished



 Best hyperparameters:
{'gamma': 0.1, 'C': 4}


Performance Matrix for Support Vector Classifier with Hyper Paramter tuning:
Accuracy =  0.7054973821989529
F1 Score =  0.7678018575851393
AUC Score = 0.6938679472118601
Kappa score =  0.3963761500105345


Creating function for oversampling with hyper-paramter tuning

In [0]:
def oversample_HP(X, y): 
    from imblearn.over_sampling import SMOTE 
    sm = SMOTE() 
    X, y = sm.fit_sample(X, y)
    
    from sklearn.feature_selection import f_classif
    from sklearn.feature_selection import SelectKBest 
    fs = SelectKBest(f_classif, k = 8)
    X = fs.fit_transform(X, y)
    
    #Splitting the data into train test spit
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 50)
    
    from sklearn.naive_bayes import GaussianNB
    gnb = GaussianNB()
    gnb.fit(X_train, y_train)

    # Predicting the Test set results
    y_pred = gnb.predict(X_test)
    print("Performance Matrix for Gaussain Naive Bayes:")
    performance_matrix(y_test, y_pred)

    print("\n\n HyperParamter-Tuning XGBoost\n\n")
    from xgboost import XGBClassifier
    xgb = XGBClassifier(tree_method = 'gpu_hist', subsample = 1.0, n_estimators = 86, min_child_weight = 1, max_depth = 4, learning_rate = 0.5, eta = 0.11)
    hyperparamter_tuning_XGB(X_train, y_train)
    xgb.fit(X_train, y_train)   
    # Predicting the Test set results
    y_pred = xgb.predict(X_test)
    print("\n\nPerformance Matrix for XGBoost Classifier with Hyper Paramter tuning:")
    performance_matrix(y_test, y_pred)
    
    
    print("\n\n HyperParamter-Tuning Random Forest \n\n")
    from sklearn.ensemble import RandomForestClassifier
    rf = RandomForestClassifier(n_estimators = 1620, min_samples_leaf = 1, max_depth = 100)
    hyperparamter_tuning_RF(X_train, y_train)
    rf.fit(X_train, y_train)
    # Predicting the Test set results
    y_pred = rf.predict(X_test)
    print("\n\nPerformance Matrix for RandomForest Classifier with Hyper Paramter tuning:")
    performance_matrix(y_test, y_pred)
    
    print("\n\n HyperParamter-Tuning Support Vector Machine \n\n")
    from sklearn.svm import SVC
    hyperparamter_tuning_SVM(X_train, y_train)
    svc = SVC(random_state = 10, gamma = 0.1, C = 4)
    svc.fit(X_train, y_train)
    y_pred = svc.predict(X_test)
    print("\n\nPerformance Matrix for Support Vector Classifier with Hyper Paramter tuning:")
    performance_matrix(y_test, y_pred)

Initializing oversampling with hyper-paramter tuning function

In [20]:
oversample_HP(X, y)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Performance Matrix for Gaussain Naive Bayes:
Accuracy =  0.7756964457252642
F1 Score =  0.7590299277605779
AUC Score = 0.7747119725939833
Kappa score =  0.550429702247426


 HyperParamter-Tuning XGBoost


Fitting 5 folds for each of 100 candidates, totalling 500 fits


/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:   10.1s
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:   31.1s
[Parallel(n_jobs=-1)]: Done 284 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  1.9min finished



 Best hyperparameters:
{'subsample': 1.0, 'n_estimators': 86, 'min_child_weight': 1, 'max_depth': 4, 'learning_rate': 0.5, 'eta': 0.11}


Performance Matrix for XGBoost Classifier with Hyper Paramter tuning:
Accuracy =  0.9279538904899135
F1 Score =  0.9265066144047035
AUC Score = 0.9278969758997359
Kappa score =  0.8558524128104418


 HyperParamter-Tuning Random Forest 


Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  6.1min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed: 27.0min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 49.1min finished




 Best hyperparameters:
{'n_estimators': 370, 'min_samples_leaf': 1, 'max_depth': 70}


Performance Matrix for RandomForest Classifier with Hyper Paramter tuning:
Accuracy =  0.9245917387127762
F1 Score =  0.9252024773701762
AUC Score = 0.9250289902396642
Kappa score =  0.8492748223512462


 HyperParamter-Tuning Support Vector Machine 


Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed: 10.5min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 13.9min finished



 Best hyperparameters:
{'gamma': 0.1, 'C': 4}


Performance Matrix for Support Vector Classifier with Hyper Paramter tuning:
Accuracy =  0.7961095100864554
F1 Score =  0.8217509972706277
AUC Score = 0.798918274639396
Kappa score =  0.5944207958412723
